In [1]:
import tensorflow as tf
from probabilistic_vq.vector_quantizer import VectorQuantizer

In [2]:
class Model(tf.keras.models.Model):
    
    def __init__(self, **kwargs):
        """
        """
        tf.keras.models.Model.__init__(self, **kwargs)
        
        self.conv1 = tf.keras.layers.Conv2D(
            32, 5, strides=(2, 2), padding='same')
        self.conv2 = tf.keras.layers.Conv2D(
            64, 5, strides=(2, 2), padding='same')
        self.conv3 = tf.keras.layers.Conv2D(
            128, 5, strides=(2, 2), padding='same')
        
        self.vq = VectorQuantizer(10, 128)
        
        self.conv1_transpose = tf.keras.layers.Conv2DTranspose(
            64, 5, strides=(2, 2), padding='same')
        self.conv2_transpose = tf.keras.layers.Conv2DTranspose(
            32, 5, strides=(2, 2), padding='same')
        self.conv3_transpose = tf.keras.layers.Conv2DTranspose(
            3, 5, strides=(2, 2), padding='same')
        
    def call(self, images, **kwargs):
        """
        """
        x = self.conv1(images, **kwargs)
        x = self.conv2(tf.nn.relu(x), **kwargs)
        x = self.conv3(tf.nn.relu(x), **kwargs)
        
        z, z_log_probs = self.vq(x, **kwargs)
        
        x = self.conv1_transpose(tf.nn.relu(z), **kwargs)
        x = self.conv2_transpose(tf.nn.relu(x), **kwargs)
        x = self.conv3_transpose(tf.nn.relu(x), **kwargs)
        return tf.nn.tanh(x), z_log_probs

In [3]:
model = Model()

In [4]:
images = tf.random.uniform([8, 32, 32, 3], minval=-1., maxval=1.)

In [5]:
y, z_log_p = model(images)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]